In [1]:
import numpy as np
import tensorflow as tf


In [2]:
ALPHABET = "abcdefghijklmnopqrstuvwxyz" # list of character
MAX_WORD_LEN = 15 # longest word our network can parse
ONE_HOT_SIZE = len(ALPHABET) * MAX_WORD_LEN # how long our one hot array should be

# get position of a char in alphabet
def getPos(char):
    if len(char) > 1:
        raise Exception("char should be a single character, length was: {}".format(len(char)))
    char = char.lower()
    if char not in ALPHABET:
        raise Exception("char should be in the alphabet, char was: {}".format(char))
    return ALPHABET.index(char)

# create a one hot array for value, length of length
def oneHot(val, length):
    arr = [0]*length
    arr[val] = 1
    return arr

# get one hot array for single char
def charOneHot(char):
    val = getPos(char)
    return oneHot(val, len(ALPHABET))

# get one hot arrays for each char in a string
def oneHotString(string):
    strLen = len(string)
    result = [0]*strLen
    for i in range(0, strLen):
        result[i] = charOneHot(string[i])
        
    return result

# transform a word to a one hot array 
def wordToInput(word):
    if len(word) > MAX_WORD_LEN:
        raise Exception("Word is too long, must be < 15 char, actual: {}".format(len(word)))
    result = [0]*ONE_HOT_SIZE
    index = 0
    for char in word:
        result[getPos(char) + index] = 1
        index = index + len(ALPHABET)
    return result
    
# great, now we can turn words into one hot arrays
# now we have to get and label training data

# functions for data wrangling

# returns true if word only contains letters in alphabet
def isAlpha(word):
    word = word.lower()
    for char in word:
        if char not in ALPHABET:
            return False
        
    return True
    

In [3]:
LANGUAGES = ["English", "German", "French"]
MIN_WORD_LEN = 3

# dictionary source: https://github.com/hermitdave/FrequencyWords

# data wrangling time, things we have to do:
# remove words shorter than 3(not required but it'll help) 
# remove words containing words outside of the english alphabet (this means no accents)
# remove words longer than 15 characters (or whatever we decide our maximum to be)
# the dataset I'm using also has frequency for each word, we don't need that

# parsing dictionaries
english = open("data/english_dict.txt", "r", encoding = "utf-8")

english_lines = english.readlines()
english_words = []
for line in english_lines:
    split = line.split(" ")
    word = split[0]
    if len(word) > MIN_WORD_LEN and len(word) < MAX_WORD_LEN and isAlpha(word):
        english_words.append(word)        
        
german = open("data/german_dict.txt", "r", encoding = "utf-8")

german_lines = german.readlines()
german_words = []
for line in german_lines:
    split = line.split(" ")
    word = split[0]
    if len(word) > MIN_WORD_LEN and len(word) < MAX_WORD_LEN and isAlpha(word):
        german_words.append(word)
        
french = open("data/french_dict.txt", "r", encoding = "utf-8")

french_lines = french.readlines()
french_words = []
for line in french_lines:
    split = line.split(" ")
    word = split[0]
    if len(word) > MIN_WORD_LEN and len(word) < MAX_WORD_LEN and isAlpha(word):
        french_words.append(word)
        
print("Word counts before slice:")
print("English words: {}".format(len(english_words)))
print("German words: {}".format(len(german_words)))
print("French words: {} \n".format(len(french_words)))


#balancing
SAMPLES = 33000 # number of words for each language, should be less than smallest dictionary

english_words = english_words[0:SAMPLES]
german_words = german_words[0:SAMPLES]
french_words = french_words[0:SAMPLES]

print("Word counts after slice:")
print("English words: {}".format(len(english_words)))
print("German words: {}".format(len(german_words)))
print("French words: {}".format(len(french_words)))



Word counts before slice:
English words: 44295
German words: 39692
French words: 33678 

Word counts after slice:
English words: 33000
German words: 33000
French words: 33000


In [14]:
# time to label data
from sklearn.utils import shuffle


# converting words to one hot encoding
X_train = []
for word in english_words:
    X_train.append(wordToInput(word))

for word in german_words:
    X_train.append(wordToInput(word))
    
for word in french_words:
    X_train.append(wordToInput(word))

X_train = np.array(X_train)

# labeling data
y_train = []
y_train.extend([0]*SAMPLES) #English
y_train.extend([1]*SAMPLES) #German
y_train.extend([2]*SAMPLES) #French

# converting labels to one hot
y_train = tf.keras.utils.to_categorical(y_train)

y_train = np.array(y_train)

# shuffling data
X_train, y_train = shuffle(X_train, y_train)
# data labeled in np array


# Validation slice

NUM_VALIDATE = 5000

X_test = X_train[len(X_train) - NUM_VALIDATE:len(X_train)]
y_test = y_train[len(y_train) - NUM_VALIDATE:len(y_train)]

X_train = X_train[0:len(X_train) - NUM_VALIDATE]
y_train = y_train[0:len(y_train) - NUM_VALIDATE]


In [19]:
# serializing
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

In [20]:
# use this cell to load serialized data
import pickle
X_train = pickle.load(open("X.pickle", "rb"))
y_train = pickle.load(open("y.pickle", "rb"))

In [51]:
# build and train model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

LAYER_SIZE = [32, 64, 128]
DROPOUT = [0, 0.33, 0.5]
HIDDEN = [1, 2]




for size in LAYER_SIZE:
    for drop in DROPOUT:
        for hidden in HIDDEN:

            model = Sequential()
            model.add(tf.keras.layers.Dense(ONE_HOT_SIZE))
            model.add(Activation("relu"))
            for h in range(0, hidden):
                model.add(tf.keras.layers.Dense(size))
                model.add(Activation("relu"))
                model.add(Dropout(drop))


            model.add(tf.keras.layers.Dense(len(LANGUAGES))) # output layer
            model.add(Activation("softmax"))

            model.compile(loss='categorical_crossentropy',
                         optimizer = 'adam',
                         metrics = ['accuracy'])
            print("training model: L{}-D{}-H{}".format(size, drop, hidden))
            model.fit(X_train, y_train, batch_size = 64, validation_split = 0.0, epochs = 10, verbose = 1)
            NAME = "models/English-German-French-L{}-D{}-H{}.model".format(size, drop, hidden)
            model.save(NAME)
            print("evaluate model: L{}-D{}-H{}".format(size, drop, hidden))
            model.evaluate(X_test, y_test)

training model: L32-D0-H1
Epoch 1/10
94000/94000 [==============================] - 6s 64us/sample - loss: 0.8512 - acc: 0.5915
Epoch 2/10
94000/94000 [==============================] - 6s 62us/sample - loss: 0.7336 - acc: 0.6557
Epoch 3/10
94000/94000 [==============================] - 6s 68us/sample - loss: 0.6860 - acc: 0.6757
Epoch 4/10
94000/94000 [==============================] - 6s 61us/sample - loss: 0.6550 - acc: 0.6883
Epoch 5/10
94000/94000 [==============================] - 6s 62us/sample - loss: 0.6319 - acc: 0.6977
Epoch 6/10
94000/94000 [==============================] - 6s 63us/sample - loss: 0.6120 - acc: 0.7044
Epoch 7/10
94000/94000 [==============================] - 6s 60us/sample - loss: 0.5960 - acc: 0.7092
Epoch 8/10
94000/94000 [==============================] - 5s 58us/sample - loss: 0.5826 - acc: 0.7143
Epoch 9/10
94000/94000 [==============================] - 6s 64us/sample - loss: 0.5704 - acc: 0.7174
Epoch 10/10
94000/94000 [==============================]

94000/94000 [==============================] - 7s 77us/sample - loss: 0.8425 - acc: 0.5941
Epoch 2/10
94000/94000 [==============================] - 7s 72us/sample - loss: 0.7278 - acc: 0.6574
Epoch 3/10
94000/94000 [==============================] - 7s 72us/sample - loss: 0.6814 - acc: 0.6774
Epoch 4/10
94000/94000 [==============================] - 7s 72us/sample - loss: 0.6496 - acc: 0.6895
Epoch 5/10
94000/94000 [==============================] - 7s 72us/sample - loss: 0.6259 - acc: 0.6977
Epoch 6/10
94000/94000 [==============================] - 7s 72us/sample - loss: 0.6048 - acc: 0.7049
Epoch 7/10
94000/94000 [==============================] - 7s 72us/sample - loss: 0.5885 - acc: 0.7096
Epoch 8/10
94000/94000 [==============================] - 7s 72us/sample - loss: 0.5741 - acc: 0.7150
Epoch 9/10
94000/94000 [==============================] - 7s 74us/sample - loss: 0.5634 - acc: 0.7177
Epoch 10/10
94000/94000 [==============================] - 7s 74us/sample - loss: 0.5513 - ac

Epoch 1/10
94000/94000 [==============================] - 10s 108us/sample - loss: 0.8548 - acc: 0.5881
Epoch 2/10
94000/94000 [==============================] - 10s 109us/sample - loss: 0.7369 - acc: 0.6536
Epoch 3/10
94000/94000 [==============================] - 11s 112us/sample - loss: 0.6868 - acc: 0.6742
Epoch 4/10
94000/94000 [==============================] - 10s 108us/sample - loss: 0.6545 - acc: 0.6873
Epoch 5/10
94000/94000 [==============================] - 10s 112us/sample - loss: 0.6297 - acc: 0.6961
Epoch 6/10
94000/94000 [==============================] - 10s 107us/sample - loss: 0.6095 - acc: 0.7022
Epoch 7/10
94000/94000 [==============================] - 10s 107us/sample - loss: 0.5932 - acc: 0.7078
Epoch 8/10
94000/94000 [==============================] - 10s 110us/sample - loss: 0.5797 - acc: 0.7121
Epoch 9/10
94000/94000 [==============================] - 10s 110us/sample - loss: 0.5692 - acc: 0.7149
Epoch 10/10
94000/94000 [==============================] - 10s 1

In [44]:
model.evaluate(X_test, y_test)

5000/5000 [==============================] - 0s 56us/sample - loss: 0.7823 - acc: 0.6404


[0.7823201487541199, 0.6404]

In [36]:
model = tf.keras.models.load_model(NAME)

In [45]:
# change the word to test
word = "wiggly"

word_vec = np.array(wordToInput(word)).reshape(-1, ONE_HOT_SIZE)
pred = model.predict(word_vec)

print("Guess: {}".format(LANGUAGES[np.argmax(pred)]))

print("Confidence: {}%".format(pred[0][np.argmax(pred)]*100))



Guess: English
Confidence: 99.62353110313416%
